## Temporal Filtering of UNOSAT Damage Assessment Data

This notebook automates the extraction of the temporally closest damage observation per site from a multi-date UNOSAT damage assessment dataset. The source data, structured as a GeoJSON file, includes point-based damage features with repeated temporal observations across multiple fields (`SensorDate`, `SensorDate_2`, ..., `SensorDate_9`). The objective is to isolate the date of observation that is closest to a given reference date — in this case, **February 18, 2025** — for each individual damage point.

---

### Methodology

1. **Data Loading and Inspection**
   - The notebook loads the GeoJSON file using `geopandas`, identifies all date-related columns, and ensures they are parsed as datetime objects.

2. **Closest Date Identification**
   - For each feature, the script:
     - Iterates over all valid observation dates
     - Computes the absolute time difference from the target date
     - Selects the date with the smallest difference
   - The result is stored in a new column, `closest_date`.

3. **Temporal Filtering (Optional)**
   - A subset of features can be extracted that fall within a specified time window around the target date (e.g., ±1 day).

4. **Data Export**
   - The filtered dataset is exported as a new GeoJSON file for visualization or further analysis in GIS environments.

---

### Applications

This workflow is useful for:
- **Conflict damage monitoring**
- **Time-sensitive impact analysis**
- Selecting representative damage records for a known **reference event or strike date**
- Reducing multi-temporal complexity into a single-date representation for mapping and statistics

---

### Technologies Used

- Python 3
- `geopandas`, `pandas`, `datetime`
- Jupyter Notebook or Google Colab

---

The notebook ensures reproducible and scalable preprocessing of UNOSAT damage data for spatiotemporal filtering, enabling targeted analysis and integration with field observations or remote sensing datasets.


In [1]:
# Install necessary libraries (only needed in Colab)
!pip install geopandas

# Imports
import geopandas as gpd
import pandas as pd
from datetime import datetime

In [3]:
# Upload your filefrom
from google.colab import drive
drive.mount('/content/drive')

# Load GeoJSON
gdf = gpd.read_file("/content/drive/MyDrive/UNOSAT_damage_pts.geojson")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Identify all SensorDate columns
date_columns = [col for col in gdf.columns if col.startswith("SensorDate")]

In [5]:
# Convert all date columns to datetime
for col in date_columns:
    gdf[col] = pd.to_datetime(gdf[col], errors='coerce')

In [6]:
# Extract the most recent (max) date per row
def most_recent_date(row):
    valid_dates = [row[col] for col in date_columns if pd.notnull(row[col])]
    return max(valid_dates) if valid_dates else pd.NaT

gdf["most_recent_date"] = gdf.apply(most_recent_date, axis=1)

In [7]:
# Optional: Drop rows with no dates at all
gdf = gdf[gdf["most_recent_date"].notnull()]

In [8]:
# Save to GeoJSON
output_file = "UNOSAT_most_recent.geojson"
gdf.to_file(output_file, driver="GeoJSON")

In [9]:
# Download the result
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>